#Paquetes necesarios

In [1]:
import cv2  
import math 

from ultralytics import YOLO

Modelos preentrenados, visualizando con las utilidades de ultralytics

In [ ]:
# Carga del modelo
#model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolo11n-seg.pt') #Máscaras
model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()

Desde cámara, detección con yolo11, modelo nano. Visualización propia con OpenCV

In [ ]:
# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Captura desde la webcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Detecta en la imagen
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Seguimiento. Requiere instalar lap con pip install lap

In [5]:
from collections import defaultdict
import numpy as np

# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car"]


# Captura desde la webcam
filename = "TGC23_PdH_C0056cut.mp4"
vid = cv2.VideoCapture(filename)
track_history = defaultdict(lambda: [])
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Seguimiento, con persistencia entre fotogramas
        results = model.track(img, persist=True, classes = [0,1,2])

        if 1:
            if results is not None:
                print(results[0])
                boxes = results[0].boxes.xywh.cpu()
                if results[0].boxes.id is not None:
                    track_ids = results[0].boxes.id.int().cpu().tolist()
                else:
                    track_ids = []
                annotated_frame = results[0].plot()
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))
                    if len(track) > 30:
                        track.pop(0)
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
                cv2.imshow("YOLO11 Tracking", annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
        

        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                #Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()




0: 384x640 4 persons, 46.0ms
Speed: 2.5ms preprocess, 46.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 5

In [8]:
from collections import defaultdict
import numpy as np
import cv2
import math

# Carga del modelo YOLO
model = YOLO('yolo11n.pt')  # Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "car", "motorbike", "bus"]

# Captura desde el archivo de video
filename = "TGC23_PdH_C0056cut.mp4"
vid = cv2.VideoCapture(filename)

# Obtenemos las propiedades del video original (ancho, alto, FPS)
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))
fps = vid.get(cv2.CAP_PROP_FPS)

# Inicializamos el VideoWriter para guardar el video anotado
output_filename = 'video_anotado_solo_deteccion.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codificador de video
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

track_history = defaultdict(lambda: [])

while True:
    # fotograma a fotograma
    ret, img = vid.read()

    # Si hay imagen válida
    if ret:
        # Seguimiento con persistencia entre fotogramas
        results = model.track(img, persist=True, classes=[0,1,2,3])

        if results is not None:
            # Creamos una imagen en negro del mismo tamaño que el fotograma original
            img_detections = np.zeros_like(img)

            # Procesamos cada detección
            for r in results:
                boxes = r.boxes

                for box in boxes:
                    # Contenedor de cada detección
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                    # Recortar el área detectada en el fotograma original
                    detection_crop = img[y1:y2, x1:x2]

                    # Pegar el área recortada en la imagen en negro
                    img_detections[y1:y2, x1:x2] = detection_crop

                    # Dibuja un recuadro y pone la etiqueta en la imagen de detecciones
                    if box.id is not None:
                        track_id = str(int(box.id[0].tolist()))
                    else:
                        track_id = ''

                    # Confianza
                    confidence = math.ceil((box.conf[0] * 100)) / 100
                    print("Confianza --->", confidence)

                    # Clase
                    cls = int(box.cls[0])
                    print("Clase -->", classNames[cls])

                    # Dibuja el contenedor y clase en la imagen de detección
                    cv2.rectangle(img_detections, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.putText(img_detections, track_id + ' ' + classNames[cls], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Guardar el fotograma procesado en el archivo de video
            out.write(img_detections)
            cv2.imshow("Detección YOLO", img_detections)

            # Detener si se presiona la tecla "q"
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

    # Detenemos el proceso si se presiona ESC
    if cv2.waitKey(20) == 27:
        break

# Libera los recursos
vid.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 4 persons, 49.5ms
Speed: 3.0ms preprocess, 49.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.51
Clase --> person
Confianza ---> 0.43
Clase --> person
Confianza ---> 0.3
Clase --> person
Confianza ---> 0.12
Clase --> person

0: 384x640 4 persons, 47.5ms
Speed: 1.0ms preprocess, 47.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.5
Clase --> person
Confianza ---> 0.4
Clase --> person
Confianza ---> 0.31
Clase --> person
Confianza ---> 0.12
Clase --> person

0: 384x640 4 persons, 41.0ms
Speed: 1.0ms preprocess, 41.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.51
Clase --> person
Confianza ---> 0.41
Clase --> person
Confianza ---> 0.35
Clase --> person
Confianza ---> 0.12
Clase --> person

0: 384x640 4 persons, 41.5ms
Speed: 1.4ms preprocess, 41.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.51
Clase --> person
Confianza --->

In [10]:
from ultralytics import YOLO

# Cargar el modelo YOLOv8 (puedes usar uno preentrenado)
model = YOLO('yolov8n.pt')  # También puedes usar 'yolov8s.pt' para un modelo más grande

# Entrenar el modelo con tu dataset personalizado
model.train(
    data='license.yaml',  # El archivo de configuración YAML
    epochs=50,  # Ajusta el número de épocas según sea necesario
    imgsz=433,  # Tamaño de las imágenes (puedes ajustarlo)
    batch=16,   # Tamaño del batch
    device='cpu'    # Para entrenar con GPU, usa '0'. Para CPU, usa 'cpu'
)

New https://pypi.org/project/ultralytics/8.3.24 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.23  Python-3.9.20 torch-2.5.0+cpu 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: 0
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


Intregración con seguimiento (tracking)
!!!!!!!!!Nota: he tenido que bajar a la versión de python 3.9.5 e instalar lap con pip install lap

In [ ]:
# Carga del modelo
model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolov11n-seg.pt') #Máscaras
#model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model.track(source=filename, show=True)  # BoT-SORT tracker (por defecto)
#results = model.track(source=filename, show=True, tracker="bytetrack.yaml")  # ByteTrack tracker

cv2.destroyAllWindows()

Reconocimiento de caracteres tras instalar pytesseract y tesseract

In [ ]:
# Tesseract
import cv2
import pytesseract
from pytesseract import Output

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('ocr_test.tif') 

if img is not None:
    #Convierte a RGB antes de procesar
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Texto localizado
    print(pytesseract.image_to_string(img))

    #Texto y localización en imagen de cada palabra
    d = pytesseract.image_to_data(img_rgb, output_type=Output.DICT)

    n_boxes = len(d['text'])
    for i in range(n_boxes):
        #Nivel de confianza
        if int(d['conf'][i]) > 60:
            text = d['text'][i]
            conf = d['conf'][i]
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

            print(f'Texto: {text} ({conf:.2f}%)\nContenedor: {x,y,x+w,y+h}')

    cv2.imshow('img', img_rgb)
    cv2.waitKey(-1)

   

else:
    print('Error de imagen')

Reconocimiento de caracteres tras instalar easyocr

In [ ]:
import easyocr

#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
res = reader.readtext('ocr_test.tif')

for (bbox, text, prob) in res:
    # Coordenadas en orden 
    (top_left, top_right, bottom_right, bottom_left) = bbox
    print(f'\nTexto: {text}\nProbabilidad: {prob:.2f}\nContenedor: {tuple(map(int, top_left)),tuple(map(int, bottom_right))}')


#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')